# Camada Gold

In [0]:
CREATE SCHEMA IF NOT EXISTS workspace.projeto_vendas_gold;

## KPI Geral - Visão Executiva

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_gold.kpi_geral AS
SELECT
  COUNT(DISTINCT o.order_id) AS total_pedidos,
  COUNT(DISTINCT c.customer_id) AS total_clientes,
  SUM(oi.price) AS faturamento_total,
  SUM(oi.price) / COUNT(DISTINCT o.order_id) AS ticket_medio
FROM workspace.projeto_vendas_silver.orders o
JOIN workspace.projeto_vendas_silver.order_items oi
  ON o.order_id = oi.order_id
JOIN workspace.projeto_vendas_silver.customers c
  ON o.customer_id = c.customer_id;

In [0]:
SELECT * FROM workspace.projeto_vendas_gold.kpi_geral

## Faturamento por Estado

In [0]:
-- Granularidade: estado
CREATE OR REPLACE TABLE workspace.projeto_vendas_gold.faturamento_estado AS
SELECT
  c.customer_state,
  SUM(oi.price) AS faturamento
FROM workspace.projeto_vendas_silver.orders o
JOIN workspace.projeto_vendas_silver.order_items oi
  ON o.order_id = oi.order_id
JOIN workspace.projeto_vendas_silver.customers c
  ON o.customer_id = c.customer_id
GROUP BY c.customer_state
ORDER BY faturamento DESC;

In [0]:
SELECT * FROM workspace.projeto_vendas_gold.faturamento_estado

## Faturamento por Categoria

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_gold.faturamento_categoria AS
SELECT
  p.product_category,
  SUM(oi.price) AS faturamento
FROM workspace.projeto_vendas_silver.order_items oi
JOIN workspace.projeto_vendas_silver.products p
  ON oi.product_id = p.product_id
GROUP BY p.product_category
ORDER BY faturamento DESC;

In [0]:
SELECT * FROM workspace.projeto_vendas_gold.faturamento_categoria

## Ticket Médio

In [0]:
CREATE OR REPLACE TABLE workspace.projeto_vendas_gold.ticket_medio AS
SELECT
  SUM(price) / COUNT(DISTINCT order_id) AS ticket_medio
FROM workspace.projeto_vendas_silver.order_items;